In [1]:
spark

org.apache.spark.sql.SparkSession@35253ecc

In [2]:
val data = spark.read.option("header", "false").option("delimiter", "\t").csv("hdfs://mr-hn:8020/user/acald013/Datasets/ny_bikes.wkt").cache

data = [_c0: string, _c1: string ... 3 more fields]


[_c0: string, _c1: string ... 3 more fields]

In [3]:
case class Route(wkt: String, id: String, t1: Long, t2: Long, duration: Long)
//**

val trajs = data.map{ row =>
  Route(
    row.getString(0),
    row.getString(1),
    row.getString(2).toLong,
    row.getString(3).toLong,
    row.getString(4).toLong
  )
}.cache

defined class Route
trajs = [wkt: string, id: string ... 3 more fields]


[wkt: string, id: string ... 3 more fields]

In [4]:
trajs.show

+--------------------+----------------+-------------+-------------+--------+
|                 wkt|              id|           t1|           t2|duration|
+--------------------+----------------+-------------+-------------+--------+
|LINESTRING (-73.9...|703143368D531D3B|1682105347000|1682106295000| 1082305|
|LINESTRING (-73.9...|293D3CD88AB1ED0F|1682540848000|1682541439000|  509974|
|LINESTRING (-73.9...|6B6C3AD3E9F5EE64|1680598050000|1680598237000|  146011|
|LINESTRING (-73.9...|159C90AB46D5631B|1680369846000|1680371170000| 1167262|
|LINESTRING (-73.9...|F96671AD26CF236F|1680895251000|1680896105000|  875629|
|LINESTRING (-73.9...|36D52B2C5233BD68|1681373408000|1681374727000| 1320907|
|LINESTRING (-73.9...|3ED7720B9AFFDF9B|1682180019000|1682181757000|  501345|
|LINESTRING (-73.9...|BA691255AB5D2F9F|1680540874000|1680541841000|  200031|
|LINESTRING (-73.9...|D6AF8DD6D8C7260B|1681934281000|1681934453000|  196043|
|LINESTRING (-73.9...|A04ED390091E5EEB|1680966144000|1680966440000|  199948|

In [1]:
%AddJar file:///home/acald013/Spark/2.4/jars/jts-core-1.19.0.jar
%AddJar file:///home/acald013/Research/Scripts/Scala/PFlock/target/scala-2.11/pflock_2.11-0.1.0.jar

Starting download from file:///home/acald013/Spark/2.4/jars/jts-core-1.19.0.jar
Finished download of jts-core-1.19.0.jar
Using cached version of jts-core-1.19.0.jar
Starting download from file:///home/acald013/Research/Scripts/Scala/PFlock/target/scala-2.11/pflock_2.11-0.1.0.jar
Finished download of pflock_2.11-0.1.0.jar
Using cached version of pflock_2.11-0.1.0.jar


In [2]:
import org.locationtech.jts.geom._
import edu.ucr.dblab.pflock.sedona.quadtree.StandardQuadTree
import scala.io.Source

val geofactory = new GeometryFactory(new PrecisionModel(1e-10))

geofactory = org.locationtech.jts.geom.GeometryFactory@57b7c64e


org.locationtech.jts.geom.GeometryFactory@57b7c64e

In [4]:
val buffer = Source.fromFile("/home/acald013/Datasets/oilpalm.csv")
val data = buffer.getLines.map{ line =>
    val arr = line.split(",")
    val x = arr(0).toDouble
    val y = arr(1).toDouble
    
    geofactory.createPoint(new Coordinate(x,y))
}.toList
buffer.close

lastException = null
buffer = <iterator>
data = List(POINT (-8170204 1200516), POINT (-8170204 1200416), POINT (-8170104 1200416), POINT (-8170104 1200516), POINT (-8170204 1200516), POINT (-8167404 1200916), POINT (-8167204 1200916), POINT (-8167204 1200716), POINT (-8167104 1200716), POINT (-8167104 1200616), POINT (-8167004 1200616), POINT (-8167004 1200416), POINT (-8166904 1200416), POINT (-8166904 1200316), POINT (-8167004 1200316), POINT (-8167004 1200216), POINT (-8167404 1200216), POINT (-8167404 1200316), POINT (-8167504 1200316), POINT (-8167504 1200416), POINT (-8167604 1200416), POINT (-8167604 1200516), POINT (-8167404 1200516), POINT (-8167404 1200716), POINT (-8167804 1200716), POINT (-8167804 1200816), POINT (-8167404 1200816),...


List(POINT (-8170204 1200516), POINT (-8170204 1200416), POINT (-8170104 1200416), POINT (-8170104 1200516), POINT (-8170204 1200516), POINT (-8167404 1200916), POINT (-8167204 1200916), POINT (-8167204 1200716), POINT (-8167104 1200716), POINT (-8167104 1200616), POINT (-8167004 1200616), POINT (-8167004 1200416), POINT (-8166904 1200416), POINT (-8166904 1200316), POINT (-8167004 1200316), POINT (-8167004 1200216), POINT (-8167404 1200216), POINT (-8167404 1200316), POINT (-8167504 1200316), POINT (-8167504 1200416), POINT (-8167604 1200416), POINT (-8167604 1200516), POINT (-8167404 1200516), POINT (-8167404 1200716), POINT (-8167804 1200716), POINT (-8167804 1200816), POINT (-8167404 1200816),...

In [5]:
val envelope = data.foldLeft(new Envelope()){ (env, curr) => 
    env.expandToInclude(curr.getCoordinate)
    env
}

envelope = Env[-8416403.2654204 : -8135704.40990079, 781487.90174011 : 1201115.93117416]


Env[-8416403.2654204 : -8135704.40990079, 781487.90174011 : 1201115.93117416]

In [6]:
val rect = new edu.ucr.dblab.pflock.sedona.quadtree.QuadRectangle(envelope)

rect = x: -8416403.2654204 y: 781487.90174011 w: 280698.8555196095 h: 419628.02943404997 PartitionId: null Lineage: null


x: -8416403.2654204 y: 781487.90174011 w: 280698.8555196095 h: 419628.02943404997 PartitionId: null Lineage: null

In [7]:
import scala.jdk.CollectionConverters._

In [8]:
val quadtree = new StandardQuadTree[Int](rect, 0, 1000, 10)
data.foreach{ p =>
    quadtree.insert(new edu.ucr.dblab.pflock.sedona.quadtree.QuadRectangle(p.getEnvelopeInternal), 1)
}
quadtree.getLeafZones.asScala.size

quadtree = edu.ucr.dblab.pflock.sedona.quadtree.StandardQuadTree@12aaf08d


322

In [9]:
quadtree.getLeafZones.asScala.map{ r =>
    geofactory.toGeometry(r.getEnvelope).toText
}.foreach(println)

POLYGON ((-8416403 1174889, -8416403 1201116, -8398860 1201116, -8398860 1174889, -8416403 1174889))
POLYGON ((-8398860 1174889, -8398860 1201116, -8381316 1201116, -8381316 1174889, -8398860 1174889))
POLYGON ((-8416403 1148662, -8416403 1174889, -8398860 1174889, -8398860 1148662, -8416403 1148662))
POLYGON ((-8398860 1161776, -8398860 1174889, -8390088 1174889, -8390088 1161776, -8398860 1161776))
POLYGON ((-8390088 1161776, -8390088 1174889, -8381316 1174889, -8381316 1161776, -8390088 1161776))
POLYGON ((-8398860 1148662, -8398860 1161776, -8390088 1161776, -8390088 1148662, -8398860 1148662))
POLYGON ((-8390088 1148662, -8390088 1161776, -8381316 1161776, -8381316 1148662, -8390088 1148662))
POLYGON ((-8381316 1174889, -8381316 1201116, -8363772 1201116, -8363772 1174889, -8381316 1174889))
POLYGON ((-8363772 1174889, -8363772 1201116, -8346229 1201116, -8346229 1174889, -8363772 1174889))
POLYGON ((-8381316 1148662, -8381316 1174889, -8363772 1174889, -8363772 1148662, -8381316 

In [7]:
%AddJar file:///home/acald013/.cache/coursier/v1/https/repo1.maven.org/maven2/com/github/nscala-time/nscala-time_2.11/2.32.0/nscala-time_2.11-2.32.0.jar

Starting download from file:///home/acald013/.cache/coursier/v1/https/repo1.maven.org/maven2/com/github/nscala-time/nscala-time_2.11/2.32.0/nscala-time_2.11-2.32.0.jar
Finished download of nscala-time_2.11-2.32.0.jar
Using cached version of nscala-time_2.11-2.32.0.jar


In [8]:
import com.github.nscala_time.time.Imports._

trajs.rdd.map{ route =>
    val start = new DateTime(route.t1, DateTimeZone.UTC)
    val end = new DateTime(route.t2, DateTimeZone.UTC)
    
    (start, end, route.t2 - route.t1, route.duration)
}.take(10).foreach(println)

(2023-04-21T19:29:07.000Z,2023-04-21T19:44:55.000Z,948000,1082305)
(2023-04-26T20:27:28.000Z,2023-04-26T20:37:19.000Z,591000,509974)
(2023-04-04T08:47:30.000Z,2023-04-04T08:50:37.000Z,187000,146011)
(2023-04-01T17:24:06.000Z,2023-04-01T17:46:10.000Z,1324000,1167262)
(2023-04-07T19:20:51.000Z,2023-04-07T19:35:05.000Z,854000,875629)
(2023-04-13T08:10:08.000Z,2023-04-13T08:32:07.000Z,1319000,1320907)
(2023-04-22T16:13:39.000Z,2023-04-22T16:42:37.000Z,1738000,501345)
(2023-04-03T16:54:34.000Z,2023-04-03T17:10:41.000Z,967000,200031)
(2023-04-19T19:58:01.000Z,2023-04-19T20:00:53.000Z,172000,196043)
(2023-04-08T15:02:24.000Z,2023-04-08T15:07:20.000Z,296000,199948)


In [9]:
val router1 = geofactory.createPoint(new Coordinate(10.0, 7.0))
val router2 = geofactory.createPoint(new Coordinate(10.0, 7.0))
val router3 = geofactory.createPoint(new Coordinate(10.0, 7.0))
val router4 = geofactory.createPoint(new Coordinate(10.0, 7.0))



router1 = POINT (10 7)
router2 = POINT (10 7)
router3 = POINT (10 7)
router4 = POINT (10 7)


POINT (10 7)

In [10]:
val filename = "https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2023-04.parquet"
val data = spark.read.parquet(filename).cache

In [11]:
data.show

+--------------------+----------------+-------------+-------------+-------+
|                 _c0|             _c1|          _c2|          _c3|    _c4|
+--------------------+----------------+-------------+-------------+-------+
|LINESTRING (-73.9...|703143368D531D3B|1682105347000|1682106295000|1082305|
|LINESTRING (-73.9...|293D3CD88AB1ED0F|1682540848000|1682541439000| 509974|
|LINESTRING (-73.9...|6B6C3AD3E9F5EE64|1680598050000|1680598237000| 146011|
|LINESTRING (-73.9...|159C90AB46D5631B|1680369846000|1680371170000|1167262|
|LINESTRING (-73.9...|F96671AD26CF236F|1680895251000|1680896105000| 875629|
|LINESTRING (-73.9...|36D52B2C5233BD68|1681373408000|1681374727000|1320907|
|LINESTRING (-73.9...|3ED7720B9AFFDF9B|1682180019000|1682181757000| 501345|
|LINESTRING (-73.9...|BA691255AB5D2F9F|1680540874000|1680541841000| 200031|
|LINESTRING (-73.9...|D6AF8DD6D8C7260B|1681934281000|1681934453000| 196043|
|LINESTRING (-73.9...|A04ED390091E5EEB|1680966144000|1680966440000| 199948|
|LINESTRING 

lastException = null


null

In [15]:
val geofactory = new GeometryFactory(new PrecisionModel(1e10))
val envelope = new Envelope(-74.0217207046608, -73.9851479264573, 40.69775127799, 40.7196722289007)
geofactory.toGeometry(envelope).toText

geofactory = org.locationtech.jts.geom.GeometryFactory@7410e33b
envelope = Env[-74.0217207046608 : -73.9851479264573, 40.69775127799 : 40.7196722289007]


POLYGON ((-74.02172070466 40.69775127799, -74.02172070466 40.7196722289, -73.98514792646 40.7196722289, -73.98514792646 40.69775127799, -74.02172070466 40.69775127799))